In [1]:
import torch

a = torch.Tensor([2,3,4,5])


tensor([2., 3., 4., 5.])


In [6]:
b = torch.Tensor([[1,2],[3,4],[5,6]])
b.shape

torch.Size([3, 2])

In [8]:
import numpy as np

a = np.array([2,3,1,2,3])
a.tolist()

[2, 3, 1, 2, 3]

In [30]:
torch.as_tensor(np.arange(4)[np.random.binomial(1, 0.5, size=4) == 1], dtype=torch.long)

tensor([1, 3])

In [32]:
from torch_cluster import random_walk

row = torch.tensor([0, 1, 1, 1, 2, 2, 3, 3, 4, 4])
col = torch.tensor([1, 0, 2, 3, 1, 4, 1, 4, 2, 3])
start = torch.tensor([0, 1, 2, 3, 4])

walk = random_walk(row, col, start, walk_length=3)
print(walk)

tensor([[0, 1, 2, 4],
        [1, 2, 4, 3],
        [2, 4, 2, 4],
        [3, 1, 0, 1],
        [4, 3, 1, 0]])


In [44]:
b = torch.tensor([[1,2,3,3,3],[2,1,4,4,5]])


tensor([[1, 2, 3, 3, 3],
        [2, 1, 4, 4, 5]])
tensor([1, 2, 3, 3])
tensor([2, 3, 3, 3])


In [53]:
b = np.array([[1,2,3,4,5],[2,3,4,5,6]])
np.where((b[0, :] == 2) & (b[1, :] == 3))[0][0]

1

In [2]:
%load_ext autoreload
%autoreload 1
%aimport bitcoin

In [14]:
data = bitcoin.BitcoinOTC()

In [15]:
sample_str = 'random-walk'
sample_args = {'start_num': 5, 'walk_length': 50}
print(data.nodes_pos_degree_mask)
print(data.edges_pos_degree_mask)

tensor([   0,    1,    3,  ..., 5994, 5995, 5998])
tensor([    0,     1,     2,  ..., 35589, 35590, 35591])


In [4]:
len(data.pos_degree_mask)

4791

In [5]:
import torch
import numpy as np
from torch_cluster import random_walk

a = torch.tensor([2,3,4,5])
b = torch.tensor([1,4,7,1])
c = torch.tensor([1,2,3,4])

TypeError: where(): argument 'condition' (position 1) must be Tensor, not bool

In [13]:
np.where(np.isin(a.numpy(), c.numpy()) & np.isin(b.numpy(), c.numpy()))[0]

(array([0, 1], dtype=int64),)

In [19]:
start_num = 5
walk_length = 10

start_loc = torch.as_tensor(
    np.random.choice(data.nodes_pos_degree_mask.numpy(), size=start_num, replace=False),
    dtype=torch.long
)
print(start_loc)

tensor([4458, 3874, 3653, 3736, 1160])


In [83]:
# Perform random walks; note that walks has shape
# [start_num, walk_length + 1]
walks = random_walk(row=data.edge_index[0, data.edges_pos_degree_mask],
                    col=data.edge_index[1, data.edges_pos_degree_mask],
                    start=start_loc,
                    walk_length=walk_length,
                    coalesced=True)

print(walks)

tensor([[4458, 2641, 3201, 2641, 2724, 2261, 2223, 2304, 2223, 2261, 4087],
        [3874, 3597, 2457, 3597, 2062, 4196,  303, 2084, 2088,   34,   95],
        [3653, 3652, 1554, 1347, 1457, 1216, 1457, 1216, 2816, 1216, 1621],
        [3736,  435, 2298,   34, 3804,   34, 1395,   40,    6,  934, 1561],
        [1160,  519,  149,  734,  861,  797,  831,  867,  831, 1033, 4314]])


In [84]:
ind_mask = (data.edge_index[0, :] == 4458)
ind_mask2 = (data.edge_index[1, :] == 2641)
print(torch.where(ind_mask & ind_mask2))

ind_mask_rev = (data.edge_index[0, :] == 5402)
ind_mask_rev2 = (data.edge_index[1, :] == 4458)
print(torch.where(ind_mask_rev & ind_mask_rev2))

(tensor([24448]),)
(tensor([], dtype=torch.int64),)


In [85]:
# Convert into edge list
edge_list = torch.zeros(size=(2, start_num*walk_length),
                        dtype=torch.long)

for i in range(start_num):
    ind = i*walk_length + np.arange(walk_length)
    ind = list(ind)
    edge_list[0, ind] = walks[i, :-1]
    edge_list[1, ind] = walks[i, 1:]
    
print(edge_list)

tensor([[4458, 2641, 3201, 2641, 2724, 2261, 2223, 2304, 2223, 2261, 3874, 3597,
         2457, 3597, 2062, 4196,  303, 2084, 2088,   34, 3653, 3652, 1554, 1347,
         1457, 1216, 1457, 1216, 2816, 1216, 3736,  435, 2298,   34, 3804,   34,
         1395,   40,    6,  934, 1160,  519,  149,  734,  861,  797,  831,  867,
          831, 1033],
        [2641, 3201, 2641, 2724, 2261, 2223, 2304, 2223, 2261, 4087, 3597, 2457,
         3597, 2062, 4196,  303, 2084, 2088,   34,   95, 3652, 1554, 1347, 1457,
         1216, 1457, 1216, 2816, 1216, 1621,  435, 2298,   34, 3804,   34, 1395,
           40,    6,  934, 1561,  519,  149,  734,  861,  797,  831,  867,  831,
         1033, 4314]])


In [86]:
# Get rid of duplicates in edge list
edge_list = torch.unique(edge_list, dim=1)

print(edge_list)
print(edge_list.shape[1])

tensor([[   6,   34,   34,   34,   40,  149,  303,  435,  519,  734,  797,  831,
          831,  861,  867,  934, 1033, 1160, 1216, 1216, 1216, 1347, 1395, 1457,
         1554, 2062, 2084, 2088, 2223, 2223, 2261, 2261, 2298, 2304, 2457, 2641,
         2641, 2724, 2816, 3201, 3597, 3597, 3652, 3653, 3736, 3804, 3874, 4196,
         4458],
        [ 934,   95, 1395, 3804,    6,  734, 2084, 2298,  149,  861,  831,  867,
         1033,  797,  831, 1561, 4314,  519, 1457, 1621, 2816, 1457,   40, 1216,
         1347, 4196, 2088,   34, 2261, 2304, 2223, 4087,   34, 2223, 3597, 2724,
         3201, 2261, 1216, 2641, 2062, 2457, 1554, 3652,  435,   34, 3597,  303,
         2641]])
49


In [87]:
# Extract nodes and the indices of edge_list with respect to
# self.edge_index
node_ind = torch.unique(edge_list)
print(node_ind)

tensor([   6,   34,   40,   95,  149,  303,  435,  519,  734,  797,  831,  861,
         867,  934, 1033, 1160, 1216, 1347, 1395, 1457, 1554, 1561, 1621, 2062,
        2084, 2088, 2223, 2261, 2298, 2304, 2457, 2641, 2724, 2816, 3201, 3597,
        3652, 3653, 3736, 3804, 3874, 4087, 4196, 4314, 4458])


In [90]:
edge_ind = torch.zeros(size=[edge_list.shape[1]], dtype=torch.long)
print(edge_ind)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])


In [92]:
for i in range(edge_list.shape[1]):
    b = (data.edge_index[0, :] == edge_list[0, i]) & (data.edge_index[1, :] == edge_list[1, i])
    a = torch.where(b)[0]
    edge_ind[i] = a[0]
    
print(edge_ind)

tensor([ 6508, 34397,  7241, 20290,  6086,  3659, 10310, 11728,  2043,  3610,
         3810,  3575,  4326,  3617,  3576,  7777, 32423,  5001,  6694,  7529,
        16576,  6682,  7334,  6695,  7569, 35363, 10738, 10278, 12065, 12046,
        12097, 22026, 11778, 11964, 12716, 17505, 16936, 15446, 16512, 16937,
        18310, 12715, 19962, 19449, 20510, 20294, 23033, 29604, 24448])


In [94]:
subsample_dict = {'node_ind': node_ind, 'edge_ind': edge_ind,
                  'edge_list': edge_list}

print(subsample_dict)

{'node_ind': tensor([   6,   34,   40,   95,  149,  303,  435,  519,  734,  797,  831,  861,
         867,  934, 1033, 1160, 1216, 1347, 1395, 1457, 1554, 1561, 1621, 2062,
        2084, 2088, 2223, 2261, 2298, 2304, 2457, 2641, 2724, 2816, 3201, 3597,
        3652, 3653, 3736, 3804, 3874, 4087, 4196, 4314, 4458]), 'edge_ind': tensor([ 6508, 34397,  7241, 20290,  6086,  3659, 10310, 11728,  2043,  3610,
         3810,  3575,  4326,  3617,  3576,  7777, 32423,  5001,  6694,  7529,
        16576,  6682,  7334,  6695,  7569, 35363, 10738, 10278, 12065, 12046,
        12097, 22026, 11778, 11964, 12716, 17505, 16936, 15446, 16512, 16937,
        18310, 12715, 19962, 19449, 20510, 20294, 23033, 29604, 24448]), 'edge_list': tensor([[   6,   34,   34,   34,   40,  149,  303,  435,  519,  734,  797,  831,
          831,  861,  867,  934, 1033, 1160, 1216, 1216, 1216, 1347, 1395, 1457,
         1554, 2062, 2084, 2088, 2223, 2223, 2261, 2261, 2298, 2304, 2457, 2641,
         2641, 2724, 2816, 320

In [97]:
data = bitcoin.BitcoinOTC()

sample_str = 'random-walk'
sample_args = {'start_num': 5, 'walk_length': 50}

test = data.subsample(sample_str=sample_str, sample_args=sample_args)
print(test)

{'node_ind': tensor([   0,    3,    6,   16,   18,   34,   40,   55,   56,   59,  103,  134,
         140,  142,  143,  170,  180,  208,  213,  215,  226,  229,  246,  379,
         395,  400,  416,  461,  473,  492,  494,  521,  522,  534,  544,  545,
         575,  591,  629,  646,  651,  686,  708,  713,  728,  734,  756,  766,
         792,  793,  845,  855,  861,  881,  907,  921,  927,  985, 1008, 1012,
        1017, 1047, 1093, 1118, 1244, 1266, 1316, 1333, 1351, 1362, 1393, 1395,
        1411, 1501, 1554, 1561, 1564, 1565, 1574, 1582, 1583, 1584, 1618, 1636,
        1638, 1692, 1713, 1724, 1734, 1753, 1801, 1809, 1818, 1839, 1893, 1898,
        1899, 1952, 1976, 1977, 2016, 2018, 2027, 2044, 2066, 2134, 2158, 2279,
        2290, 2295, 2305, 2321, 2343, 2484, 2485, 2502, 2634, 2641, 2657, 2680,
        2709, 2724, 2726, 2727, 2777, 2847, 2857, 2859, 2867, 2873, 2908, 2909,
        2933, 2979, 3001, 3017, 3145, 3173, 3184, 3218, 3220, 3226, 3253, 3282,
        3365, 3403, 3464, 3